The dataset from the grasp_diffusion package is a modified version of the ACRONYM dataset.
In this notebook the structure of the dataset is analysed.

In [3]:
from pathlib import Path
import random
import math

import trimesh
import h5py

In [4]:
DATASET_PATH = Path("/home/data")

In [6]:
set([p.name for p in (DATASET_PATH / "grasps").iterdir()]) == set(
    [p.name for p in (DATASET_PATH / "meshes").iterdir()]
)
# -> both directories have the same folders

True

In [ ]:
for object_name in [p.name for p in (DATASET_PATH / "grasps").iterdir()]:
    for filename in (DATASET_PATH / "grasps" / object_name).iterdir():

In [ ]:
def load_mesh(grasp_file_path):
    grasp_data = h5py.File(grasp_file_path, 'r')
    mesh_scale = grasp_data["object"]["scale"][()]
    mesh_file_path = (
        MESHES_PATH / grasp_data["object"]["file"][()].decode("utf-8")[len("meshes") + 1 :]
    )
    mesh = trimesh.load_mesh(mesh_file_path)
    if type(mesh) == trimesh.scene.scene.Scene:
        mesh = trimesh.util.concatenate(mesh.dump())
    mesh = mesh.apply_translation(-mesh.centroid)
    mesh = mesh.apply_scale(mesh_scale)
    # turn 90 degrees around x axis
    mesh = mesh.apply_transform(trimesh.transformations.rotation_matrix(
        angle=-math.pi/2, direction=[1, 0, 0], point=[0, 0, 0]
    ))

    return mesh

In [ ]:
N_OBJECTS = 10

random.seed(42)
grasp_file_paths = random.sample(list(GRASPS_PATH.rglob("*.h5")), N_OBJECTS)

meshes = [load_mesh(grasp_file_path) for grasp_file_path in grasp_file_paths]

In [ ]:
for mesh, path in zip(meshes, grasp_file_paths):
    print(path.parts[-2], mesh.bounds[1] - mesh.bounds[0], mesh.units)

In [ ]:
SIZE = 1

scene = trimesh.Scene()
scene.add_geometry(trimesh.creation.axis(axis_length=SIZE))

for i, mesh in enumerate(meshes):
    mesh = mesh.copy().apply_translation([(i + 1) * SIZE, 0, 0])
    scene.add_geometry(mesh)
    scene.add_geometry(
        create_gripper_marker(scale=1.0)
        .apply_translation([(i + 1.5) * SIZE, 0, 0])
        .apply_transform(
            trimesh.transformations.rotation_matrix(
                angle=-math.pi/2, direction=[1, 0, 0], point=[0, 0, 0]
            )
        )
    )

scene.show()